# 🔹 What is Apache Spark?

Apache Spark is an open-source distributed computing system used for big data processing and analytics. It’s known for its speed, ease of use, and ability to process large-scale data.

### Key Concepts:
**Distributed computing:** Spark processes data in parallel across many machines.

**In-memory processing:** Unlike Hadoop, Spark can keep data in memory for faster access.

**Unified engine:** Supports multiple tasks like batch processing, streaming, machine learning, and SQL.

### Spark Ecosystem Components:

| **Component**     | **Description** |
|-------------------|-----------------|
| **Spark SQL**     | Run SQL queries on large datasets. Supports structured and semi-structured data. |
| **Spark Core**    | The foundation – handles basic I/O, scheduling, and task execution. |
| **Spark Streaming** | For processing real-time data streams. |


# 🔹 Parallelism
### 🍬 Real-Life Analogy: Counting M&M Lentils with Spark

Imagine you're organizing a big event and you receive a **huge bag of mixed M&M lentils**—millions of them.

Your goal? 👉 Count how many M&Ms there are of each color.

You don't want to do this alone—it would take forever. So, you call your friends (Spark Cluster) to help you out!

---

#### 🧠 The Spark Cluster Analogy

| **Spark Component** | **Real-Life Equivalent** |
|---------------------|--------------------------|
| **Driver Program**  | You (the event organizer giving instructions) |
| **Executors**       | Your friends helping count M&Ms |
| **Cluster Manager** | Your group chat deciding who does what |
| **Tasks**           | "Count the red ones from this bucket" |
| **Partitions**      | Separate bowls of M&Ms to divide the work |

---

#### 🛠️ How the Work is Distributed

1. **Driver Program (You)**:  
   You write down a plan: "Count how many red, blue, green, yellow M&Ms there are."

2. **Divide the Work (Partition the Data)**:  
   You pour the big bag into 4 bowls (data partitions), one for each friend.

3. **Send Tasks to Executors**:  
   You give each friend a simple task:  
   "Count how many of each color you find in your bowl."

4. **Friends Start Counting (Parallel Processing)**:  
   All friends count at the same time – no waiting!  
   Each friend returns a small result like:  
   - Friend 1: Red – 105, Blue – 87, Green – 94  
   - Friend 2: Red – 98, Blue – 102, Green – 101  
   *(...and so on)*

5. **Combine the Results (Aggregation at Driver)**:  
   You collect all the partial counts and add them together:  
   - Total Red = 105 + 98 + ...  
   - Total Blue = 87 + 102 + ...

---

#### ✅ Spark Benefits Shown Here:

- **Speed**: Counting is done in parallel.
- **Scalability**: You can invite more friends (add executors) if the bag gets bigger.
- **Efficiency**: You only do the final summation (aggregation).

---

#### 📌 In Real Spark SQL

You might run:
```sql
SELECT color, COUNT(*) FROM m_and_ms GROUP BY color


# 🔹 Partitioning (Legacy) & Liquid Clustering (Modern)
---
So how can we help with efficiency and scalability within code?  
Traditionally, the first answer was **proper table partitioning**.  

### 🧩 Table Partitioning: Distributing Work Evenly

Continuing our M&M analogy 🍬...

Let’s say you again have 4 friends helping to count M&Ms. But this time, you accidentally pour 90% of the M&Ms into **one** bowl, and only small amounts into the other 3.

Now what happens?

- One friend is **overwhelmed** with M&Ms. 😩  
- The other three finish quickly and **sit idle**. 🪑  
- The final result takes much longer than necessary. 🕒  

This is called **data skew** — when some partitions have much more data than others.

---

### 🔍 What is Table Partitioning?

In Spark (and Delta Lake / Parquet), **partitioning** means physically organizing data files **by column values**, typically in the file system (e.g., `/table/date=2025-07-18/`).  

Partitioning allows Spark to:

- **Prune unnecessary data** (read only what’s needed)  
- **Parallelize processing efficiently**  
- **Avoid scanning full tables**  

---

### 🧠 Good Partitioning Example

If you’re querying a table by **`event_date`** most of the time, you could partition by `event_date`.

That way, when running:

```sql
SELECT * FROM events WHERE event_date = '2025-07-18'


### ⚠️ What Happens with Bad Partitioning?

Let’s explore **two common partitioning mistakes**:

---

#### ❌ 1. Partitioning by a Skewed Column

Let’s say you partition by `user_id`, and one user (e.g., `user_42`) has 90% of the activity.

- Spark writes most of the data into one partition (`user_id=42`).  
- That partition takes much longer to process.  
- Other partitions are idle, causing **data skew** and **slow jobs**.  

---

#### ❌ 2. Partitioning by a Unique Key (e.g., Primary Key)

Let’s say you partition by `transaction_id` or `order_id` — which are **unique for each row**.

- Spark creates **one folder per row** (partition).  
- You end up with **millions of small files** and folders.  
- Metadata operations become slow (e.g., listing files, updating table state).  
- Queries are slower because Spark must scan lots of tiny files.  
- Writes can fail or severely degrade due to **filesystem overhead**.  

📉 This is called the **small file problem** — and it kills performance at scale.

---

### 🧠 Key Insight

> **Just because a column is important (like a primary key) doesn’t mean it should be used for partitioning.**

---

### ✅ Better Alternatives (Legacy Approach)

| Instead of this...                | Try this...                           |
|----------------------------------|---------------------------------------|
| Partitioning by `transaction_id` | Don’t partition, or use `event_date`  |
| Partitioning by `user_id`        | Use `country`, `region`, or `user_type` if more balanced |
| No partitioning at all           | Partition by columns in common filters (e.g., `event_date`, `category`) |

---

# 🔹 How Liquid Clustering Helps
---

With **Liquid Clustering**, these pitfalls are much less of a problem:  

- Clustering doesn’t create **one folder per value** — data stays in large Delta files, reorganized internally.  
- Skewed values (like a hot `user_id`) are handled more gracefully by **incremental reclustering**.  
- No more **partition explosion** from unique keys — you can safely include higher-cardinality columns (e.g., `user_id`) in the clustering keys.  
- You can **change clustering keys later** if query patterns evolve, without rewriting the whole dataset.  

---

### 🧠 Modern Key Insight

> **Partitioning needs careful design.  
> Clustering gives you flexibility.**  

For new Delta tables:  
- Avoid over-partitioning.  
- Use **`CLUSTER BY`** for most scenarios.  
- Let **Automatic Clustering** pick keys if you don’t want to. 🤖  

---

### 📦 Summary

- Partitioning helps **divide the work**, but poor partitioning creates more problems than it solves.  
- **Liquid Clustering** reduces the risk of skew and small-file issues while still giving you efficient query pruning.  
- Spark loves **“just right” data layout** — today, that usually means **clustering** instead of heavy manual partitioning.  


# 🔹 Exercise 1: Partitioning vs Clustering

In this exercise we’ll see **why partitioning choice matters**, and how **Liquid Clustering** provides a modern alternative.

---

### 📝 Steps:
1. Create a sample dataframe / temporary view.  
2. Create 2 **partitioned tables**:  
   - ❌ Partitioned by a primary key (bad practice)  
   - ✅ Partitioned by `event_date` (better practice)  
3. Create 1 **liquid clustered table**:  
   - 🚀 Clustered by (`event_date`)  
4. Check table definitions.  
5. Clean up resources.  

---

### 📊 Expected Outcome

- The table **partitioned by primary key** will:  
  - Generate many tiny files/folders.  
  - Write slowly and create metadata overhead.  

- The table **partitioned by `event_date`** will:  
  - Write more efficiently.  
  - Allow pruning by date filters.  

- The table with **Liquid Clustering** will:  
  - Avoid the small-file problem.  
  - Support flexible future query patterns.  
  - Perform well even with higher-cardinality clustering keys.  

---

### ⚡ Modern Key Insight

> Partitioning can be good when used carefully, but **Liquid Clustering is the recommended default** for new Delta tables.  


In [0]:
%python
from pyspark.sql.functions import expr
from datetime import datetime

# Create demo DataFrame
data = spark.range(0, 10000).selectExpr(
    "id as order_id",
    "CAST(id % 50 AS STRING) AS user_id",  # 50 users
    "date_sub(current_date(), CAST(id % 10 AS INT)) AS event_date",
    "CAST(rand() * 100 AS DECIMAL(10,2)) AS amount"
)

data.createOrReplaceTempView("tmp_sales")

In [0]:

CREATE TABLE exercise1_training_table_bad_partitioning 
USING DELTA
PARTITIONED BY (order_id) -- partitioned by primary Key ( big problem)
AS 
SELECT
  *
FROM
  tmp_sales;

In [0]:
DESCRIBE TABLE exercise1_training_table_bad_partitioning;

In [0]:
CREATE TABLE exercise1_training_table_good_partitioning
USING DELTA
PARTITIONED BY (event_date) -- partitioned by event_date (well distributed across workers, alows to be pruned when filtered by where statement)
AS
SELECT * FROM tmp_sales;

In [0]:
DESCRIBE table exercise1_training_table_good_partitioning;

In [0]:
CREATE TABLE exercise1_training_table_liquid_clustering
USING DELTA
CLUSTER BY (event_date)  -- modern approach
AS
SELECT * FROM tmp_sales

In [0]:
DESCRIBE DETAIL exercise1_training_table_liquid_clustering

In [0]:
DROP TABLE IF EXISTS exercise1_training_table_good_partitioning;
DROP TABLE IF EXISTS exercise1_training_table_bad_partitioning;
DROP TABLE IF EXISTS exercise1_training_table_liquid_clustering;

### 🧠 What Do We See? 

- The **well-partitioned table** (by `event_date`) took significantly **less time to create and query** than the poorly partitioned table.  
- The **number of files** written to storage tells the story:  
  - ❌ Bad partitioned table (by primary key, e.g., `order_id`):  
    - 10,000 rows → 10,000 partitions → 10,000 Parquet files  
    - Every query must scan/load all those tiny files → slow performance, high metadata overhead  
  - ✅ Well-partitioned table (by `event_date`):  
    - 10,000 rows → ~10 partitions → fewer, larger files → efficient reads  

- 🚀 **Liquid Clustering** table:  
  - Still creates **a manageable number of files**, but organizes data **internally by clustering key** (`event_date`)  
  - Avoids small-file explosion even with higher cardinality columns  
  - Query pruning works similarly to partitions but with **more flexibility**  

- 📝 Key takeaway:  
  - **Bad partitioning = many tiny files = slow queries**  
  - **Good partitioning = fewer, balanced files = faster queries**  
  - **Liquid clustering = modern approach**: flexible, efficient, less maintenance overhead  

> In short: Spark works best when the data layout is “just right” — not too many partitions, not too few, and ideally clustered for common query patterns.  

# 🔹 BONUS 

---

# 🚀 Delta Optimization & Z-Ordering in Databricks vs Liquid clustering 

After partitioning your tables, there's more you can do to **improve performance**, especially for:
- **Large tables**
- **Frequent queries on specific columns**

---

### 🔧 What is `OPTIMIZE`?

Delta Lake tables can accumulate many small files over time (especially from writes or partitions).

The `OPTIMIZE` command **compacts small files into larger ones**, which:
- Reduces metadata overhead
- Speeds up reads and filters
- Improves shuffle performance

---

### 🧭 What is `ZORDER`?

Z-Ordering is like sorting — but for multiple dimensions.  
It **co-locates related data** in files, even within the same partition.

> Think of ZORDER like **clustering**: putting similar records close together on disk.

You should ZORDER on **columns used in WHERE filters or joins**.

**Example:**
```sql
OPTIMIZE my_table ZORDER BY (user_id, event_date);
```
---
### 💧 What is Liquid Clustering (`CLUSTER BY AUTO`)?

Liquid Clustering is **automatic clustering** introduced in Delta Lake to reduce manual ZORDER maintenance.  

It:
- Dynamically **reorganizes data** based on query patterns
- Uses **column statistics and query workload** to cluster data automatically
- Works well for **columns that frequently appear in filters or joins**, without explicitly specifying them

**Example:**
```sql
CLUSTER BY AUTO
```

---
### ⚖️ ZORDER vs Liquid Clustering

| Feature                     | ZORDER                         | Liquid Clustering (AUTO)            |
|-------------------------------|--------------------------------|------------------------------------|
| Manual vs Automatic          | Manual selection of columns    | Automatic selection based on queries |
| Use case                     | Known filter/join columns      | Unknown or evolving query patterns |
| Maintenance                  | Must update ZORDER periodically | Self-maintaining                   |
| Performance                  | Excellent if columns chosen well | Often comparable or better over time |
| Complexity                   | Higher (requires tuning)       | Lower (hands-off)                  |
